In [410]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from StatusMsg import StatusMsg
import json
import os
import io
import datetime
import timedelta

In [411]:
def india(state,date):
    # URL = "https://www.mygov.in/corona-data/covid19-statewise-status/"
    # file_name, headers = urllib.request.urlretrieve(URL)
    soup = BeautifulSoup(open("../INPUT/"+date+"/TT_State.html", encoding="utf8"), "html.parser")

    # sum_URL = "https://www.mygov.in/covid-19"
    # sum_file_name, sum_headers = urllib.request.urlretrieve(sum_URL)
    sum_soup = BeautifulSoup(open("../INPUT/"+date+"/TT.html", encoding="utf8"), "html.parser")

    STATES = soup.find_all("div", {"class": "field field-name-field-select-state field-type-list-text field-label-above"})
    CONFIRMED = soup.find_all("div", {"class": "field field-name-field-total-confirmed-indians field-type-number-integer field-label-above"})
    CURED_DISCHARGED = soup.find_all("div", {"class": "field field-name-field-cured field-type-number-integer field-label-above"})
    DEATH = soup.find_all("div", {"class": "field field-name-field-deaths field-type-number-integer field-label-above"})

    states = []
    for val in STATES:
        states.append(str(val.getText()).split(":")[1].lstrip().replace("Telengana", "Telangana").replace('Andaman And Nicobar', 'Andaman and Nicobar Islands').replace('Andaman and Nicobar','Andaman and Nicobar Islands'))

    confirmed = []
    for val in CONFIRMED:
        confirmed.append(str(val.getText()).split(":")[1].lstrip())

    cured = []
    for val in CURED_DISCHARGED:
        cured.append(str(val.getText()).split(":")[1].lstrip())

    death = []
    for val in DEATH:
        death.append(str(val.getText()).split(":")[1].lstrip())

    states_data = pd.DataFrame(list(zip(states, confirmed, cured, death)))

    final_df_col = [
        'Date', 'State/UTCode', 'District', 'tested_last_updated_district', 'tested_source_district',
        'notesForDistrict', 'cumulativeConfirmedNumberForDistrict', 'cumulativeDeceasedNumberForDistrict',
        'cumulativeRecoveredNumberForDistrict', 'cumulativeTestedNumberForDistrict',
        'cumulativeVaccinatedNumberForDistrict', 'last_updated', 'tested_last_updated_state', 'tested_source_state',
        'notesForState', 'cumulativeConfirmedNumberForState', 'cumulativeDeceasedNumberForState',
        'cumulativeRecoveredNumberForState', 'cumulativeTestedNumberForState', 'cumulativeVaccinatedNumberForState'
    ]

    states_data = states_data.rename(
        columns={0: "District", 1: "cumulativeConfirmedNumberForDistrict", 2: "cumulativeRecoveredNumberForDistrict",
                 3: "cumulativeDeceasedNumberForDistrict"})

    states_data["Date"] = date#str(datetime.datetime.now().date())

    states_data["State/UTCode"] = state

    states_data['cumulativeConfirmedNumberForState'] = int(
        sum_soup.findAll("div", {"class": "t_case"})[0].findAll("span", {"class": "icount"})[0].getText().replace(
            ",", ""))

    states_data['cumulativeDeceasedNumberForState'] = int(
        sum_soup.findAll("div", {"class": "death_case"})[0].findAll("span", {"class": "icount"})[0].getText().replace(
            ",", ""))

    states_data['cumulativeRecoveredNumberForState'] = int(
        sum_soup.findAll("div", {"class": "discharge"})[0].findAll("span", {"class": "icount"})[0].getText().replace(
            ",", ""))
    
    
    states_data["last_updated"] = str(datetime.datetime.now())

    states_data['cumulativeTestedNumberForState'] = int(
        sum_soup.findAll("div", {"class": "testing_result"})[0].findAll("strong")[0].getText().replace(",", ""))
    
    
    states_data['tested_last_updated_state'] = sum_soup.findAll(
        "div", {"class": "test_title"})[0].getText().strip().split("up to ")[-1]
    
    states_data['deltaTestedForState'] = int(
        sum_soup.findAll("div", {"class": "testing_sample"})[0].findAll("strong")[0].getText().replace(",", ""))
    
    for vcount in sum_soup.findAll("div", {"class": "total-vcount"}):
        if 'yday' not in vcount.attrs['class']:
            states_data['cumulativeVaccinatedNumberForState'] = int(vcount.findAll("strong")[0].getText().replace(",", ""))

    # states_data = states_data.reindex(columns=final_df_col)
    return states_data

In [412]:
#today = (datetime.datetime.now() - timedelta.Timedelta(days=0)).date()
#pDate = (datetime.datetime.now() - timedelta.Timedelta(days=1)).date()
#print(str(today))
#print(str(pDate))
#TT_df = india("TT",str(today))

# ASHA .. For Running previous dates
date_time_str = '2021/11/02'
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y/%m/%d')
print(str(date_time_obj.date()))
today = (date_time_obj - timedelta.Timedelta(days=0)).date()
pDate = (date_time_obj - timedelta.Timedelta(days=1)).date()
print(str(today))
print(str(pDate))
TT_df = india("TT",str(today))
# ASHA .. For Running previous dates



2021-11-02
2021-11-02
2021-11-01


In [413]:
if not os.path.isdir(os.path.join("..","RAWCSV",str(pDate),"myGov")):
    os.mkdir(os.path.join("..","RAWCSV",str(pDate),"myGov"))
    print("Created")

In [414]:
#mygov of prevDate
from ExtractStateMyGov import ExtractStateMyGov
source = pd.read_csv(r"../sources.csv")
for idx in source.index:
    if source["StateCode"][idx] != "TT":
        if source["myGov"][idx] == "yes":
            temp_df = ExtractStateMyGov(source["StateCode"][idx],str(today))
        else:
            temp_df = ExtractStateMyGov(source["StateCode"][idx],str(today), no_source = True)
        temp_df["Date"] = pDate
        temp_df.to_csv(os.path.join("..","RAWCSV",str(pDate),"myGov",source["StateCode"][idx]+"_raw.csv"))

/home/swiadmin/Incovid19/incovid19/src/ExtractStateMyGov.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df["cumulativeConfirmedNumberForState"] = state_df['cumulativeConfirmedNumberForDistrict'][state_df['District'] == 'Unknown'] = int(ind["cumulativeConfirmedNumberForDistrict"][ind["District"].str.contains(state_name)].values[0])
/home/swiadmin/Incovid19/incovid19/src/ExtractStateMyGov.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df["cumulativeDeceasedNumberForState"] = state_df['cumulativeDeceasedNumberForDistrict'][state_df['District'] == 'Unknown'] = int(ind["cumulativeDeceasedNumberForDis

In [415]:
# TT_df = pd.read_csv("../RAWCSV/2021-10-31/TT_raw.csv")

In [416]:
TT_df = TT_df.dropna(1)

In [417]:
import urllib.request, json 
with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(today)) as url:
    data = json.loads(url.read().decode())

In [418]:
TT_df["cumulativeVaccinated1NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_1"]
TT_df["cumulativeVaccinated2NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_2"]
TT_df["cumulativeVaccinatedNumberForState"] = data["topBlock"]["vaccination"]["total_doses"]

In [419]:
states = []
vaccinated = []
vaccinated1 = []
vaccinated2 = []
for sta in data['getBeneficiariesGroupBy']:
    if (sta['title'].lower() == "dadra and nagar haveli") or (sta['title'].lower() == "daman and diu"):
        states.append('Dadra and Nagar Haveli and Daman and Diu')
    else:
        states.append(sta['title'])
    vaccinated.append(sta['total'])
    vaccinated1.append(sta['partial_vaccinated'])
    vaccinated2.append(sta['totally_vaccinated'])

In [420]:
VAC_df = pd.DataFrame(list(zip(states,vaccinated,vaccinated1,vaccinated2))).groupby(0, as_index=False).sum()

In [421]:
VAC_df = VAC_df.rename(columns={0:"District",1:"cumulativeVaccinatedNumberForDistrict",2:"cumulativeVaccinated1NumberForDistrict",3:"cumulativeVaccinated2NumberForDistrict",})

In [422]:
TT_df = TT_df.merge(VAC_df,how="left",on="District")

In [423]:
TT_df["tested_source_state"] = "https://www.icmr.gov.in/"

In [424]:
# TT_df.to_csv("TT_test.csv")

In [425]:
TT_df["tested_last_updated_state"] = str(datetime.datetime.strptime(TT_df["tested_last_updated_state"][0] , '%b %d, %Y'))

# Generate TT_final

In [426]:
# cs = pd.read_csv("case_time_series.csv")

In [427]:
# cs.columns

In [428]:
# cs.tail()

In [429]:
# import timedelta
# delta_date = str((datetime.datetime.now() - timedelta.Timedelta(days=1)).date())

In [430]:
delta_date = str(pDate)

In [431]:
delta_date

'2021-11-01'

In [432]:
# ind = cs[cs['Date_YMD']==delta_date].index[0]

# Integrate is CS only

In [433]:
# states_df = pd.read_csv("states.csv")

In [434]:
# cs = states_df

In [435]:
# ind = cs[(cs['Date']==delta_date) & (cs["State"] == "India")].index[0]

In [436]:
# ind

# Reading from Raw

In [437]:
cs = pd.read_csv("../RAWCSV/"+delta_date+"/TT_final.csv")

In [438]:
# cs.columns

In [439]:
# TT_df["deltaConfirmedForState"] = TT_df["cumulativeConfirmedNumberForState"] - cs["Confirmed"][ind]
# TT_df["deltaDeceasedForState"] = TT_df["cumulativeDeceasedNumberForState"] - cs["Deceased"][ind]
# TT_df["deltaRecoveredForState"] = TT_df["cumulativeRecoveredNumberForState"] - cs["Recovered"][ind]
TT_df["deltaConfirmedForState"] = TT_df["cumulativeConfirmedNumberForState"] - cs["cumulativeConfirmedNumberForState"][0]
TT_df["deltaDeceasedForState"] = TT_df["cumulativeDeceasedNumberForState"] - cs["cumulativeDeceasedNumberForState"][0]
TT_df["deltaRecoveredForState"] = TT_df["cumulativeRecoveredNumberForState"] - cs["cumulativeRecoveredNumberForState"][0]
TT_df["deltaTestedForState"] = TT_df["cumulativeTestedNumberForState"] - cs["cumulativeTestedNumberForState"][0]

In [440]:
import urllib.request, json 
with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+delta_date) as url:
    data = json.loads(url.read().decode())

In [441]:
# TT_df["cumulativeVaccinated1NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_1"]
# TT_df["cumulativeVaccinated2NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_2"]
# TT_df["cumulativeVaccinatedNumberForState"] = data["topBlock"]["vaccination"]["total_doses"]

In [442]:
TT_df["deltaVaccinated2ForState"] = TT_df["cumulativeVaccinated2NumberForState"] - data["topBlock"]["vaccination"]["tot_dose_2"]
TT_df["deltaVaccinated1ForState"] = TT_df["cumulativeVaccinated1NumberForState"] - data["topBlock"]["vaccination"]["tot_dose_1"]
TT_df["deltaVaccinatedForState"] = TT_df["cumulativeVaccinatedNumberForState"] - data["topBlock"]["vaccination"]["total_doses"]

In [443]:
# TT_df["deltaVaccinatedForState"]

In [444]:
states = []
vaccinated = []
vaccinated1 = []
vaccinated2 = []
for sta in data['getBeneficiariesGroupBy']:
    if (sta['title'].lower() == "dadra and nagar haveli") or (sta['title'].lower() == "daman and diu"):
        states.append('Dadra and Nagar Haveli and Daman and Diu')
    else:
        states.append(sta['title'])
    vaccinated.append(sta['total'])
    vaccinated1.append(sta['partial_vaccinated'])
    vaccinated2.append(sta['totally_vaccinated'])

In [445]:
prevdayVACC_df = pd.DataFrame(list(zip(states,vaccinated,vaccinated1,vaccinated2))).groupby(0, as_index=False).sum()

In [446]:
prevdayVACC_df = prevdayVACC_df.rename(columns={0:"District",1:"cumulativeVaccinatedNumberForDistrict",2:"cumulativeVaccinated1NumberForDistrict",3:"cumulativeVaccinated2NumberForDistrict",})

In [447]:
# TT_df.to_csv("in_test.csv")

In [448]:
lst = ["Vaccinated1","Vaccinated2","Vaccinated"]
for val in lst:
    TT_df["delta{}ForDistrict".format(val)] = None
    for idx in TT_df.index:
        TT_df["delta{}ForDistrict".format(val)][idx] = TT_df["cumulative{}NumberForDistrict".format(val)][idx] - prevdayVACC_df[prevdayVACC_df["District"] == TT_df["District"][idx]]["cumulative{}NumberForDistrict".format(val)].item()

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [449]:
# states_df = pd.read_csv("states.csv")

# No reading states

In [450]:
states_df = cs[["Date","District","cumulativeConfirmedNumberForDistrict","cumulativeRecoveredNumberForDistrict","cumulativeDeceasedNumberForDistrict"]]

In [451]:
states_df

,Date,District,cumulativeConfirmedNumberForDistrict,cumulativeRecoveredNumberForDistrict,cumulativeDeceasedNumberForDistrict
0,2021-11-01,Andaman and Nicobar Islands,7651,7518,129
1,2021-11-01,Andhra Pradesh,2066450,2047722,14373
2,2021-11-01,Arunachal Pradesh,55155,54774,280
3,2021-11-01,Assam,610645,600974,5997
4,2021-11-01,Bihar,726098,716390,9661
5,2021-11-01,Chandigarh,65351,64495,820
6,2021-11-01,Chhattisgarh,1006052,992159,13577
7,2021-11-01,Dadra and Nagar Haveli and Daman and Diu,10682,10674,4
8,2021-11-01,Delhi,1439870,1414431,25091
9,2021-11-01,Goa,178108,174392,3364


In [452]:
# deltaConfirmedForDistrict",
#                  "deltaDeceasedForDistrict",
#                  "deltaRecoveredForDistrict",
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    TT_df["delta{}ForDistrict".format(val)] = None
    for idx in TT_df.index:
        # print(TT_df["cumulative{}NumberForDistrict".format(val)][idx])
        # print(TT_df["District".format(val)][idx])
        # print(delta_date)
        # print(int(TT_df["cumulative{}NumberForDistrict".format(val)][idx]) - (states_df[(states_df["District"] == TT_df["District"][idx]) & (states_df["Date"] == delta_date)]["cumulative{}NumberForDistrict".format(val)])) - cumulative{}NumberForDistrict
        TT_df["delta{}ForDistrict".format(val)][idx] = int(TT_df["cumulative{}NumberForDistrict".format(val)][idx]) - int(states_df[(states_df["District"] == TT_df["District"][idx]) & (states_df["Date"] == delta_date)]["cumulative{}NumberForDistrict".format(val)].item())

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [453]:
lst = ["Vaccinated1","Vaccinated2","Vaccinated"]
for val in lst:
    TT_df["delta{}ForDistrict".format(val)] = None
    for idx in TT_df.index:
        TT_df["delta{}ForDistrict".format(val)][idx] = TT_df["cumulative{}NumberForDistrict".format(val)][idx] - prevdayVACC_df[prevdayVACC_df["District"] == TT_df["District"][idx]]["cumulative{}NumberForDistrict".format(val)].item()

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [454]:
# TT_df 7dma Cummulative

In [455]:
cs.tail()

,Unnamed: 0,District,cumulativeConfirmedNumberForDistrict,cumulativeRecoveredNumberForDistrict,cumulativeDeceasedNumberForDistrict,Date,State/UTCode,cumulativeConfirmedNumberForState,cumulativeDeceasedNumberForState,cumulativeRecoveredNumberForState,last_updated,cumulativeTestedNumberForState,tested_last_updated_state,deltaTestedForState,cumulativeVaccinatedNumberForState,cumulativeVaccinated1NumberForState,cumulativeVaccinated2NumberForState,cumulativeVaccinatedNumberForDistrict,cumulativeVaccinated1NumberForDistrict,cumulativeVaccinated2NumberForDistrict,tested_source_state,deltaConfirmedForState,deltaDeceasedForState,deltaRecoveredForState,deltaVaccinated2ForState,deltaVaccinated1ForState,deltaVaccinatedForState,deltaVaccinated1ForDistrict,deltaVaccinated2ForDistrict,deltaVaccinatedForDistrict,deltaConfirmedForDistrict,deltaRecoveredForDistrict,deltaDeceasedForDistrict,7DmaConfirmedForState,7DmaRecoveredForState,7DmaDeceasedForState,7DmaVaccinated1ForState,7DmaVaccinated2ForState,7DmaVaccinatedForState,7DmaConfirmedForDistrict,7DmaRecoveredForDistrict,7DmaDeceasedForDistrict,7DmaVaccinatedForDistrict,7DmaVaccinated1ForDistrict,7DmaVaccinated2ForDistrict,7DmaTestedForState,delta21_14confirmedForState,statePopulation,StateName,districtPopulation
31,31,Telangana,671463,663498,3956,2021-11-01,TT,34285814,458437,33668560,2021-11-11 11:25:44.608876,609201294,2021-10-31 00:00:00,881379,1068796182,736128383,332667799,32580088,22666905,9913183,https://www.icmr.gov.in/,12514,251,12718,3160106,2195076,5355182,169417,141302,310719,121,183,1,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363440e+06,1.340161e+06,16386.142857,0,0,0,NaN,NaN,1332900000,Telangana,37220000
32,32,Tripura,84504,83545,816,2021-11-01,TT,34285814,458437,33668560,2021-11-11 11:25:44.608876,609201294,2021-10-31 00:00:00,881379,1068796182,736128383,332667799,4165233,2510531,1654702,https://www.icmr.gov.in/,12514,251,12718,3160106,2195076,5355182,2055,33721,35776,33,16,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363440e+06,1.340161e+06,16386.142857,0,0,0,NaN,NaN,1332900000,Tripura,3992000
33,33,Uttar Pradesh,1710158,1687151,22900,2021-11-01,TT,34285814,458437,33668560,2021-11-11 11:25:44.608876,609201294,2021-10-31 00:00:00,881379,1068796182,736128383,332667799,131707494,98537599,33169895,https://www.icmr.gov.in/,12514,251,12718,3160106,2195076,5355182,361818,491346,853164,6,6,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363440e+06,1.340161e+06,16386.142857,0,0,0,NaN,NaN,1332900000,Uttar Pradesh,224979000
34,34,Uttarakhand,343896,336345,7400,2021-11-01,TT,34285814,458437,33668560,2021-11-11 11:25:44.608876,609201294,2021-10-31 00:00:00,881379,1068796182,736128383,332667799,11430294,7482278,3948016,https://www.icmr.gov.in/,12514,251,12718,3160106,2195076,5355182,4279,49877,54156,5,9,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363440e+06,1.340161e+06,16386.142857,0,0,0,NaN,NaN,1332900000,Uttarakhand,11141000
35,35,West Bengal,1592908,1565471,19141,2021-11-01,TT,34285814,458437,33668560,2021-11-11 11:25:44.608876,609201294,2021-10-31 00:00:00,881379,1068796182,736128383,332667799,78863112,56987338,21875774,https://www.icmr.gov.in/,12514,251,12718,3160106,2195076,5355182,803933,317620,1121553,914,913,15,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363440e+06,1.340161e+06,16386.142857,75690099,54769821,20920278,NaN,NaN,1332900000,West Bengal,96906000


In [456]:
cs = pd.read_csv("case_time_series.csv")

In [457]:
states_df = pd.read_csv("states.csv")

# Remenber to read CS and states

In [458]:
from datetime import datetime, timedelta
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    value = 0
    for i in range(7):
        if i == 0:
            value = TT_df["cumulative{}NumberForState".format(val)]
        else:
            prev_date=datetime.strptime("2021-10-31","%Y-%m-%d")+timedelta(days=(-1*i))
            ind = cs[cs['Date_YMD']==str(prev_date.date())].index[0]
            value =+ cs["Total {}".format(val)][ind]
    TT_df["7Dma{}ForState".format(val)] = value/7

In [459]:
import urllib.request, json 
va1 = 0
va2 = 0
va = 0
for i in range(7):
    prev_date=datetime.strptime("2021-10-31","%Y-%m-%d")+timedelta(days=(-1*i))
    with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(prev_date.date())) as url:
        data = json.loads(url.read().decode())
        va1 += data["topBlock"]["vaccination"]["tot_dose_1"]
        va2 += data["topBlock"]["vaccination"]["tot_dose_2"]
        va += data["topBlock"]["vaccination"]["total_doses"]


In [460]:
TT_df["7DmaVaccinated1ForState"] = va1
TT_df["7DmaVaccinated2ForState"] = va2
TT_df["7DmaVaccinatedForState"] = va

In [461]:
# 7dma District

In [462]:
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    value = 0
    for idx in TT_df.index:
        for i in range(7):
            if i == 0:
                value = int(TT_df["cumulative{}NumberForDistrict".format(val)][0])
            else:
                prev_date=datetime.strptime("2021-10-31","%Y-%m-%d")+timedelta(days=(-1*i))
                value += int(states_df[(states_df["State"] == TT_df["District"][idx]) & (states_df["Date"] == str(prev_date.date()))]['{}'.format(val)].item())
    TT_df["7Dma{}ForDistrict".format(val)] = value/7          

In [463]:
cowin = pd.read_csv("cowin_vaccine_data_statewise.csv")

In [464]:
va1 = 0
va2 = 0
va = 0
TT_df["7DmaVaccinatedForDistrict"] = 0
TT_df["7DmaVaccinated1ForDistrict"] = 0
TT_df["7DmaVaccinated2ForDistrict"] = 0
for idx in TT_df.index:
    for i in range(7):
        if i == 0:
            va = int(TT_df["cumulativeVaccinatedNumberForDistrict"][idx])
            va1 = int(TT_df["cumulativeVaccinated1NumberForDistrict"][idx])
            va2 = int(TT_df["cumulativeVaccinated2NumberForDistrict"][idx])
        else:
            prev_date=datetime.strptime("2021/10/31","%Y/%m/%d")+timedelta(days=(-1*i))
            va += int(cowin[(cowin["Updated On"] ==  prev_date.strftime('%d/%m/%Y')) & (cowin["State"] == TT_df["District"][idx])]["Total Doses Administered"].item())
            va1 += int(cowin[(cowin["Updated On"] ==  prev_date.strftime('%d/%m/%Y')) & (cowin["State"] == TT_df["District"][idx])]["First Dose Administered"].item())
            va2 += int(cowin[(cowin["Updated On"] ==  prev_date.strftime('%d/%m/%Y')) & (cowin["State"] == TT_df["District"][idx])]["Second Dose Administered"].item())
TT_df["7DmaVaccinatedForDistrict"][idx] = va/7
TT_df["7DmaVaccinated1ForDistrict"][idx] = va1/7
TT_df["7DmaVaccinated2ForDistrict"][idx] = va2/7

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [465]:
TT_df["7DmaTestedForState"] = None
TT_df["delta21_14confirmedForState"] = None

In [466]:
population = pd.read_csv("../CSV/StatePopulation.csv")

In [467]:
sources = pd.read_csv("../sources.csv")

In [468]:
population = population.merge(sources,how="left",left_on = "State", right_on = "StateCode")

In [473]:
population

,State,StatePop,StateName,StateCode,StateDataSourceType,myGov,StateDataURL,Twitter Handle,Twitter Search Term
0,AN,397000,Andaman and Nicobar Islands,AN,html,yes,https://www.mygov.in/covid-19,NaN,NaN
1,AP,52221000,Andhra Pradesh,AP,html,NaN,https://hmfw.ap.gov.in/covid_dashboard.aspx,NaN,NaN
2,AR,1504000,Arunachal Pradesh,AR,Image(Twitter),NaN,https://twitter.com/DirHealth_ArPr,DirHealth_ArPr,#ArunachalCoronaUpdate
3,AS,34293000,Assam,AS,html,yes,https://www.mygov.in/covid-19,NaN,NaN
4,BR,119520000,Bihar,BR,Image(Twitter),NaN,https://twitter.com/BiharHealthDept,BiharHealthDept,#COVIDー19 Updates Bihar
5,CH,1179000,Chandigarh,CH,html,yes,https://www.mygov.in/covid-19,NaN,NaN
6,CT,28724000,Chhattisgarh,CT,Image(Twitter),NaN,https://twitter.com/HealthCgGov,HealthCgGov,#ChhattisgarhFightsCorona
7,DL,19814000,Delhi,DL,html,yes,https://www.mygov.in/covid-19,NaN,NaN
8,DN,959000,Dadra and Nagar Haveli and Daman and Diu,DN,html,yes,https://www.mygov.in/covid-19,NaN,NaN
9,GA,1540000,Goa,GA,html,yes,https://www.mygov.in/covid-19,NaN,NaN


In [474]:
population[["StateName","StatePop"]]

,StateName,StatePop
0,Andaman and Nicobar Islands,397000
1,Andhra Pradesh,52221000
2,Arunachal Pradesh,1504000
3,Assam,34293000
4,Bihar,119520000
5,Chandigarh,1179000
6,Chhattisgarh,28724000
7,Delhi,19814000
8,Dadra and Nagar Haveli and Daman and Diu,959000
9,Goa,1540000


In [475]:
TT_df["statePopulation"] = population[population["StateName"] == "India"]["StatePop"].item()

In [477]:
TT_df

,District,cumulativeConfirmedNumberForDistrict,cumulativeRecoveredNumberForDistrict,cumulativeDeceasedNumberForDistrict,Date,State/UTCode,cumulativeConfirmedNumberForState,cumulativeDeceasedNumberForState,cumulativeRecoveredNumberForState,last_updated,cumulativeTestedNumberForState,tested_last_updated_state,deltaTestedForState,cumulativeVaccinatedNumberForState,cumulativeVaccinated1NumberForState,cumulativeVaccinated2NumberForState,cumulativeVaccinatedNumberForDistrict,cumulativeVaccinated1NumberForDistrict,cumulativeVaccinated2NumberForDistrict,tested_source_state,deltaConfirmedForState,deltaDeceasedForState,deltaRecoveredForState,deltaVaccinated2ForState,deltaVaccinated1ForState,deltaVaccinatedForState,deltaVaccinated1ForDistrict,deltaVaccinated2ForDistrict,deltaVaccinatedForDistrict,deltaConfirmedForDistrict,deltaRecoveredForDistrict,deltaDeceasedForDistrict,7DmaConfirmedForState,7DmaRecoveredForState,7DmaDeceasedForState,7DmaVaccinated1ForState,7DmaVaccinated2ForState,7DmaVaccinatedForState,7DmaConfirmedForDistrict,7DmaRecoveredForDistrict,7DmaDeceasedForDistrict,7DmaVaccinatedForDistrict,7DmaVaccinated1ForDistrict,7DmaVaccinated2ForDistrict,7DmaTestedForState,delta21_14confirmedForState,statePopulation
0,Andaman and Nicobar Islands,7654,7518,129,2021-11-02,TT,34296237,458880,33683581,2021-11-12 19:36:55.656023,610210339,2021-11-01 00:00:00,1009045,1073298407,737884266,335414141,496307,294178,202129,https://www.icmr.gov.in/,10423,443,15021,2746342,1755883,4502225,81,904,985,3,0,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363440e+06,1.340161e+06,16386.142857,0,0,0,None,None,1332900000
1,Andhra Pradesh,2066670,2048151,14377,2021-11-02,TT,34296237,458880,33683581,2021-11-12 19:36:55.656023,610210339,2021-11-01 00:00:00,1009045,1073298407,737884266,335414141,53725733,33096924,20628809,https://www.icmr.gov.in/,10423,443,15021,2746342,1755883,4502225,47873,110593,158466,220,429,4,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363440e+06,1.340161e+06,16386.142857,0,0,0,None,None,1332900000
2,Arunachal Pradesh,55161,54796,280,2021-11-02,TT,34296237,458880,33683581,2021-11-12 19:36:55.656023,610210339,2021-11-01 00:00:00,1009045,1073298407,737884266,335414141,1317323,773805,543518,https://www.icmr.gov.in/,10423,443,15021,2746342,1755883,4502225,647,4006,4653,6,22,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363440e+06,1.340161e+06,16386.142857,0,0,0,None,None,1332900000
3,Assam,610941,601309,6004,2021-11-02,TT,34296237,458880,33683581,2021-11-12 19:36:55.656023,610210339,2021-11-01 00:00:00,1009045,1073298407,737884266,335414141,28577289,20233692,8343597,https://www.icmr.gov.in/,10423,443,15021,2746342,1755883,4502225,27282,127435,154717,296,335,7,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363440e+06,1.340161e+06,16386.142857,0,0,0,None,None,1332900000
4,Bihar,726102,716397,9661,2021-11-02,TT,34296237,458880,33683581,2021-11-12 19:36:55.656023,610210339,2021-11-01 00:00:00,1009045,1073298407,737884266,335414141,68781680,50097890,18683790,https://www.icmr.gov.in/,10423,443,15021,2746342,1755883,4502225,116209,165493,281702,4,7,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363440e+06,1.340161e+06,16386.142857,0,0,0,None,None,1332900000
5,Chandigarh,65354,64500,820,2021-11-02,TT,34296237,458880,33683581,2021-11-12 19:36:55.656023,610210339,2021-11-01 00:00:00,1009045,1073298407,737884266,335414141,1477814,926717,551097,https://www.icmr.gov.in/,10423,443,15021,2746342,1755883,4502225,336,1848,2184,3,5,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363440e+06,1.340161e+06,16386.142857,0,0,0,None,None,1332900000
6,Chhattisgarh,1006074,992196,13578,2021-11-02,TT,34296237,458880,33683581,2021-11-12 19:36:55.656023,610210339,2021-11-01 00:00:00,1009045,1073298407,737884266,335414141,22318077,14892622,7425455,https://www.icmr.gov.in/,10423,443,15021,2746342,1755883,4502225,25166,45064,70230,22,

In [478]:
TT_df = TT_df.merge(population[["StateName","StatePop"]],how="left",left_on = "District", right_on = "StateName")

In [479]:
TT_df = TT_df.rename(columns={"StatePop":"districtPopulation"})

In [480]:
today

datetime.date(2021, 11, 2)

In [481]:
TT_df["Date"] = str(today)

In [482]:
TT_df.to_csv('../RAWCSV/'+str(today)+'/TT_final.csv')

In [483]:
TT_df["Date"] = str(pDate)

In [484]:
TT_df.to_csv('../RAWCSV/'+str(pDate)+'/TT_final.csv')

# Stop Here

In [409]:
# 7dma delta

In [297]:
states_df[states_df["State"] == "India"]

,Date,State,Confirmed,Recovered,Deceased,Other,Tested
1,2020-01-30,India,1,0,0,0,NaN
3,2020-02-02,India,2,0,0,0,NaN
5,2020-02-03,India,3,0,0,0,NaN
7,2020-02-14,India,3,3,0,0,NaN
11,2020-03-02,India,5,3,0,0,NaN
...,...,...,...,...,...,...,...
21523,2021-10-27,India,34231243,33606777,456418,13182,604498405.0
21560,2021-10-28,India,34245550,33619966,457223,13187,605885769.0
21597,2021-10-29,India,34259765,33633515,457774,13195,607062619.0
21634,2021-10-30,India,34272705,33648187,458219,13196,608319915.0


In [298]:
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    value = []
    for i in range(7):
        if i == 0:
            value.append(TT_df["cumulative{}NumberForState".format(val)][0])
        else:
            prev_date=datetime.strptime("2021-11-01","%Y-%m-%d")+timedelta(days=(-1*i))
            value.append(states_df[(states_df["State"] == "India") & (states_df["Date"] == str(prev_date.date()))]['{}'.format(val)].item())

In [299]:
value

[458437, 458470, 458219, 457774, 457223, 456418, 455684]